In [1]:

import pandas as pd
import googlemaps
import datetime
from datetime import timedelta

gmaps = googlemaps.Client(key='A`IzaSyC08BfStY-4KCNCe9rJS438VSvB5BC-wag')

#tripsDataFile='201402_2000Trips.csv'
stationsDataFile='201402_station_data.csv'

stations = pd.DataFrame.from_csv(stationsDataFile, index_col='station_id', encoding='utf-8')
stations.describe()
SanJoseStations = stations[stations.landmark=='San Jose']
SanFranciscoStations = stations[stations.landmark=='San Francisco']
RedwoodCityStations= stations[stations.landmark=='Redwood City']
MountainViewStations= stations[stations.landmark=='Mountain View']
PaloAltoStations= stations[stations.landmark=='Palo Alto']
type(PaloAltoStations)
# stations.loc(2)
stationLocations = {}
distanceDict={}

for landmark in stations.landmark.unique():
    landmarkStations = stations[stations.landmark == landmark]
    landmarkStations
    landmarkStationIds = landmarkStations.index.get_values()
    for originStationId in landmarkStationIds:
        originStationLocation = {'lat': stations['lat'][originStationId], 'lng':stations['long'][originStationId]}
        for destinationStationId in landmarkStationIds:
            destinationStationLocation = {'lat': stations['lat'][destinationStationId], 'lng':stations['long'][destinationStationId]}
            commute= gmaps.directions(originStationLocation, destinationStationLocation, 
                        mode="bicycling",
                          departure_time=datetime.datetime.now()  )
            distanceDict[(originStationId, destinationStationId)] = commute[0]['legs'][0]['distance']['value']


In [2]:

stationDict={}
for index, row in stations.iterrows():
        stationDict[row['name']] = index
        
# tripsDataFile='201402_2000Trips.csv'
# tripsDataWithDistanceFile='201402_2000Trips_WithDistance.csv'

tripsDataFile='201402_Trips_WithDistance.csv'
tripsDataWithDistanceFile='201402_Trips_WithBicycleDistance1.csv'

# 201402_trip_data.csv

trips = pd.DataFrame.from_csv(tripsDataFile, index_col='Trip ID', encoding='utf-8')


# trips['startStationId'] = trips['Start Station']
# trips.replace({"startStationId": stationDict})
# trips['endStationId'] = trips['End Station']
# trips.replace({"endStationId": stationDict})

trips['bicycledistance'] = trips.apply(lambda row: distanceDict[(row['startStationId'], row['endStationId'])] if (row['startStationId'], row['endStationId']) in distanceDict 
                 else None               , axis=1)


trips.to_csv(tripsDataWithDistanceFile)